In [1]:
import numpy as np
import h5py
import random
from datetime import datetime
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras import layers, models
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import pickle


# Data Preprocessing

In [2]:
file_path = './data/ROAD_dataset.h5'

In [3]:
def load_h5_data(h5_file_path, anomalies):
    """
    Load data from H5 file, including normal and anomaly data.
    """
    with h5py.File(h5_file_path, 'r') as hf:
        train_data = hf['train_data/data'][:]
        train_labels = hf['train_data/labels'][:].astype(str)

        # Replace '' and '1' with 'Normal'
        train_labels = np.where((train_labels == ''), 'Normal', train_labels)

        # Drop '1' labels from training data
        train_mask = train_labels != '1'
        train_data = train_data[train_mask]
        train_labels = train_labels[train_mask]

        # Load anomaly data
        anomaly_data, anomaly_labels = [], []
        for anomaly in anomalies:
            if anomaly != 'Normal':  # Skip 'Normal'
                group = hf[f'anomaly_data/{anomaly}']
                anomaly_data.append(group['data'][:])
                anomaly_labels.extend([anomaly] * len(group['data']))

        anomaly_data = np.concatenate(anomaly_data, axis=0) if anomaly_data else np.array([])
        anomaly_labels = np.array(anomaly_labels)

    return train_data, train_labels, anomaly_data, anomaly_labels

In [4]:
def subsample_normal_data(train_data, train_labels, num_normal_samples):
    """
    Subsample "Normal" data from the training dataset.
    """
    normal_indices = np.where(train_labels == 'Normal')[0]
    if len(normal_indices) > num_normal_samples:
        normal_indices = np.random.choice(normal_indices, num_normal_samples, replace=False)

    other_indices = np.where(train_labels != 'Normal')[0]
    all_indices = np.concatenate([normal_indices, other_indices])
    return train_data[all_indices], train_labels[all_indices]

In [5]:
def contaminate_with_anomalies(train_data, train_labels, anomaly_data, anomaly_labels, percentage_contamination, remaining_samples):
    """
    Add anomalies to the training dataset based on percentage contamination.
    """
    contaminated_data, contaminated_labels = [], []
    for anomaly, percentage in percentage_contamination.items():
        num_samples_to_add = int(remaining_samples * percentage)
        anomaly_indices = np.random.choice(
            np.where(anomaly_labels == anomaly)[0], num_samples_to_add, replace=False
        )
        contaminated_data.append(anomaly_data[anomaly_indices])
        contaminated_labels.extend([anomaly] * num_samples_to_add)

    if contaminated_data:
        contaminated_data = np.concatenate(contaminated_data, axis=0)
        contaminated_labels = np.array(contaminated_labels)

        # Append contaminated data to training data
        train_data = np.concatenate([train_data, contaminated_data], axis=0)
        train_labels = np.concatenate([train_labels, contaminated_labels], axis=0)

    return train_data, train_labels

In [6]:
def normalize(data):
    """
    Normalize the data to the range [0, 1].
    """
    normalized_data = np.zeros_like(data)
    for i, sample in enumerate(data):
        for channel in range(sample.shape[-1]):
            min_val, max_val = np.percentile(sample[..., channel], [1, 99])
            normalized = (sample[..., channel] - min_val) / (max_val - min_val + 1e-8)
            normalized_data[i, ..., channel] = np.clip(normalized, 0, 1)
    return normalized_data

In [7]:
def shuffle_and_normalize_data(train_data, train_labels):
    """
    Shuffle and normalize the training data.
    """
    shuffle_indices = np.random.permutation(len(train_data))
    train_data = train_data[shuffle_indices]
    train_labels = train_labels[shuffle_indices]

    train_data = normalize(train_data)
    return train_data, train_labels

In [8]:
def calculate_class_percentages(train_labels):
    """
    Calculate the percentages of each class in the training dataset.
    """
    unique_labels, counts = np.unique(train_labels, return_counts=True)
    return {label: count / len(train_labels) for label, count in zip(unique_labels, counts)}


In [9]:
def preprocess_lofar_data(
    h5_file_path,
    anomalies,
    percentage_contamination,
    seed=42,
    normal_percentage=None,
    total_samples=None
):
    """
    Preprocess LOFAR dataset for training and testing.
    """
    np.random.seed(seed)

    # Load data
    train_data, train_labels, anomaly_data, anomaly_labels = load_h5_data(h5_file_path, anomalies)

    # Calculate number of "Normal" samples dynamically based on percentage
    num_normal_samples = int(total_samples * normal_percentage) if normal_percentage and total_samples else len(
        np.where(train_labels == 'Normal')[0])

    # Subsample "Normal" data
    train_data, train_labels = subsample_normal_data(train_data, train_labels, num_normal_samples)

    # Add anomalies if total_samples is specified
    if total_samples:
        remaining_samples = total_samples - num_normal_samples
        train_data, train_labels = contaminate_with_anomalies(
            train_data, train_labels, anomaly_data, anomaly_labels, percentage_contamination, remaining_samples
        )

    # Shuffle and normalize data
    train_data, train_labels = shuffle_and_normalize_data(train_data, train_labels)

    # Calculate class percentages
    percentages = calculate_class_percentages(train_labels)
    print("Class Percentages in Training Data:", percentages)

    # Encode labels
    label_mapping = {label: idx for idx, label in enumerate(anomalies)}
    train_labels_encoded = np.array([label_mapping[label] for label in train_labels])

    return train_data, train_labels_encoded, percentages


In [10]:
labels = [
    'Normal',  # Represent non-anomalous data
    'oscillating_tile',
    'first_order_high_noise',
    'first_order_data_loss',
    'third_order_data_loss',
    'lightning',
    'rfi_ionosphere_reflect',
    'galactic_plane',
    'source_in_sidelobes',
    'solar_storm'
]
percentage_contamination = {
    'oscillating_tile': 0.1,
    'first_order_high_noise': 0.1,
    'first_order_data_loss': 0.1,
    'third_order_data_loss': 0.1,
    'rfi_ionosphere_reflect': 0.1,
    'lightning': 0.1,
    'galactic_plane': 0.1,
    'source_in_sidelobes': 0.1,
    'solar_storm': 0.1
}

try:
    train_data, train_labels_encoded, percentages = preprocess_lofar_data(
        file_path,
        labels,
        percentage_contamination,
        seed=42,
        normal_percentage=.1,
        total_samples=500
    )
except ValueError as e:
    print(e)


Class Percentages in Training Data: {np.str_('Normal'): np.float64(0.10989010989010989), np.str_('first_order_data_loss'): np.float64(0.0989010989010989), np.str_('first_order_high_noise'): np.float64(0.0989010989010989), np.str_('galactic_plane'): np.float64(0.0989010989010989), np.str_('lightning'): np.float64(0.0989010989010989), np.str_('oscillating_tile'): np.float64(0.0989010989010989), np.str_('rfi_ionosphere_reflect'): np.float64(0.0989010989010989), np.str_('solar_storm'): np.float64(0.0989010989010989), np.str_('source_in_sidelobes'): np.float64(0.0989010989010989), np.str_('third_order_data_loss'): np.float64(0.0989010989010989)}


In [11]:
print("Train Data Shape:", train_data.shape)
print("Train Labels Shape:", train_labels_encoded.shape)


Train Data Shape: (455, 256, 256, 4)
Train Labels Shape: (455,)


In [12]:
train_data

array([[[[7.35165377e-04, 1.67488415e-05, 1.67488415e-05,
          4.80690680e-04],
         [3.38579324e-04, 2.23072766e-06, 2.23072766e-06,
          4.28687898e-04],
         [7.67324644e-04, 0.00000000e+00, 0.00000000e+00,
          5.76364750e-04],
         ...,
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
          1.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00]],

        [[7.35165377e-04, 3.25732217e-05, 3.25732217e-05,
          4.80690680e-04],
         [3.38579324e-04, 9.03820637e-06, 9.03820637e-06,
          4.28687898e-04],
         [7.67324644e-04, 1.64581379e-05, 1.64581379e-05,
          5.76364750e-04],
         ...,
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
          1.00000000e+00],
         [0.00000000e+00, 3.47045125e-06, 3.47045125e-06,
          0.00000000e+00],
         [0.00000000e+00, 0.0000000

In [13]:
train_data_flat = train_data.reshape(train_data.shape[0], -1) 

In [14]:
# Step 1: Split into Training+Validation and Test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(train_data_flat, train_labels_encoded, test_size=0.2, random_state=42)

# Step 2: Split Training+Validation into Training and Validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)


In [15]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)



In [16]:
param_grid = {
    'n_estimators': [5, 10, 15],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Grid Search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')


In [17]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [5, 10, 15]},
             scoring='accuracy')

In [18]:
# Best Parameters and Score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Test the best model
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test Accuracy:", test_score)

Best Parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 15}
Best Score: 0.663003663003663
Test Accuracy: 0.5934065934065934


In [19]:
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Evaluation Metrics:
Accuracy: 0.59
Precision: 0.66
Recall: 0.59
F1-Score: 0.60

Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.18      0.21        11
           1       0.62      0.50      0.56        10
           2       0.64      0.64      0.64        11
           3       1.00      1.00      1.00         7
           4       0.33      0.50      0.40         8
           5       1.00      0.17      0.29         6
           6       0.92      1.00      0.96        11
           7       0.71      0.42      0.53        12
           8       0.19      0.50      0.27         6
           9       1.00      1.00      1.00         9

    accuracy                           0.59        91
   macro avg       0.67      0.59      0.58        91
weighted avg       0.66      0.59      0.60        91



In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
    ])
    return model


In [21]:
# Split into train + validation (80%) and test (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(train_data, train_labels_encoded, test_size=0.2, random_state=42)

# Further split train + validation into train (75% of 80%) and validation (25% of 80%)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Ensure labels are one-hot encoded
num_classes = len(labels)  # Number of classes

# One-hot encode labels
train_labels_onehot = to_categorical(y_train, num_classes=num_classes)
val_labels_onehot = to_categorical(y_val, num_classes=num_classes)
test_labels_onehot = to_categorical(y_test, num_classes=num_classes)


In [22]:
input_shape = (256, 256, 4)  # Height, Width, Channels
num_classes = len(labels)  # Number of classes

model = build_cnn_model(input_shape, num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical crossentropy for multiclass classification
              metrics=['accuracy'])


/Users/anniepham/Desktop/brians/Turion Project/myenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
early_stopping = EarlyStopping(
    monitor='val_loss',       # Metric to monitor (e.g., 'val_loss', 'val_accuracy')
    patience=5,               # Number of epochs with no improvement after which training will be stopped
    verbose=1,                # Verbosity level
    restore_best_weights=True # Restore the model weights from the epoch with the best monitored value
)

In [24]:
history = model.fit(
    X_train, train_labels_onehot,
    validation_data=(X_val, val_labels_onehot),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 797ms/step - accuracy: 0.1852 - loss: 2.2742 - val_accuracy: 0.4835 - val_loss: 1.6320
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 802ms/step - accuracy: 0.5870 - loss: 1.4034 - val_accuracy: 0.6154 - val_loss: 1.2964
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 759ms/step - accuracy: 0.7495 - loss: 0.9142 - val_accuracy: 0.6374 - val_loss: 1.2679
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 832ms/step - accuracy: 0.7431 - loss: 0.6938 - val_accuracy: 0.6703 - val_loss: 1.2907
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 757ms/step - accuracy: 0.8357 - loss: 0.5072 - val_accuracy: 0.7143 - val_loss: 1.1109
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 797ms/step - accuracy: 0.8589 - loss: 0.4985 - val_accuracy: 0.6593 - val_loss: 1.3046
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 758ms/step - accuracy: 0.9132 - loss: 0.3023 - val_accuracy: 0.7143 - val_loss: 1.4436
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 759ms/step - accuracy: 0.9113 - loss: 0.2667 - val_accuracy: 0.6923 - val_loss:

In [25]:
test_loss, test_accuracy = model.evaluate(X_test, test_labels_onehot, batch_size=32)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.6468 - loss: 1.3795
Test Loss: 1.3406, Test Accuracy: 0.6374


In [26]:
test_labels_onehot.shape

(91, 10)

In [27]:

# Get predictions
y_pred_probs = model.predict(X_test)  # Probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(test_labels_onehot, axis=1)  # Ground truth labels (if one-hot encoded)

# Calculate Precision, Recall, and F1-Score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step
Precision: 0.7133
Recall: 0.6374
F1-Score: 0.6369


In [28]:
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"./model/model_weights/model_weights_{current_time}.pkl"
with open(file_name, 'wb') as file:
    pickle.dump(history.history, file)
    
